In [1]:
import os
import numpy as np
import pandas as pd
import itertools

from pyhht.visualization import plot_imfs
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from bokeh.plotting import figure
from bokeh.io import show, output_notebook, export
from bokeh.layouts import row, column, gridplot
from bokeh.palettes import Dark2_5 as palette
from bokeh.plotting import figure, output_file, show
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar
from bokeh.plotting import figure
from bokeh.models.glyphs import Step

import matplotlib.pyplot as plt

from utils_cython.data_utils_c import derivative
from utils.data_utils import (csvs_merge, cumsum, step_change_point, rms, imfs_decomposition,
                              hankel_svd, correlation_coeffs, cross_correlation, 
                              fft_spectrogram, poly_coeffs, scatter3d_plot, Bearing)

%matplotlib qt
%load_ext autoreload
%autoreload 2

### Preparing data

#### Create folders for processed data and merge all .csv files of each bearing in FEMTO dataset.

In [2]:
files_info = {
    # file type identifier, columns name, columns to read.
    'acc'  : {'usecols' : [0, 1, 2, 4, 5], 'names' : ['hour', 'min', 'seg', 'h_acc', 'v_acc']},
    'temp' : {'usecols' : [0, 1, 2, 4],    'names' : ['hour', 'min', 'seg', 'temp']}
}

dataset = 'femto_dataset'
os.mkdir('data/processed_data/%s' % (dataset))

for bearing in os.listdir('data/original_data/%s/' % (dataset)):
    # Creating folders for processed data.
    os.mkdir('data/processed_data/%s/%s' % (dataset, bearing))
    
    # Merging .csv files.
    csvs_merge('data/original_data/%s/%s' % (dataset, bearing), files_info, bearing, dataset)

FileExistsError: [Errno 17] File exists: 'data/processed_data/femto_dataset'

#### Read merged files and create bearings objects.

In [3]:
bearings_to_read = ['Bearing1_1', 'Bearing1_2', 'Bearing1_3', 'Bearing1_4', 'Bearing1_5', 
                    'Bearing1_6', 'Bearing1_7', 'Bearing2_1', 'Bearing2_2', 'Bearing2_3', 
                    'Bearing2_4', 'Bearing2_5', 'Bearing2_6', 'Bearing2_7', 'Bearing3_1', 
                    'Bearing3_2', 'Bearing3_3']

dataset = 'femto_dataset'
bearings = []
for bearing_to_read in bearings_to_read:
    data = {'vib' : pd.read_csv('data/processed_data/%s/%s/acc_merged.csv' % (dataset, bearing_to_read))}
    
    # Reads 'temperature' if the data exists.
    if os.path.exists('data/processed_data/%s/temp_merged.csv' % (bearing_to_read)):
            data['temp'] = pd.read_csv('data/processed_data/%s/%s/temp_merged.csv' % (dataset, bearing_to_read))
    
    bearings.append(Bearing(name=bearing_to_read, dataset=dataset, condition=bearing_to_read[7],
                            data=data, restore_results=False))

### Pre-processing analysis.

#### FFT Spectogram.

In [ ]:
# FFT Analysis
window_size = 2560; fs=25600
for bearing in bearings:

    bearing.results['fft_spectrogram'] = {'h' :  fft_spectrogram(bearing.data['vib']['h_acc'], 
                                                                 window_size=window_size, fs=fs), 
                                          
                                          'v' :  fft_spectrogram(bearing.data['vib']['v_acc'],
                                                                 window_size=window_size, fs=fs)}

##### 3D Spectogram.

In [ ]:
 step = 6; x, y, z = bearings[0].results['fft_spectrogram']['v']
scatter3d_plot(filename='fft_spectrogram_3d.html', x=np.hstack(x)[::step], y=np.hstack(y)[::step], z=np.hstack(z)[::step])

##### 2D FFT.

In [ ]:
x, y, z = bearings[0].results['fft_spectrogram']['v']
x, y, z = x[::20], y[::20], z[::20] 
f, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharey=True, gridspec_kw={'hspace': 1})
ax1.bar(x[0], z[0], width=1.5)
ax1.set_xlabel('Frequency (Hz)')
ax1.set_ylabel('Magnitude')
ax1.set_title('Beginning')

ax2.bar(x[len(x)//2], z[len(x)//2], width=1.5)
ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('Magnitude')
ax2.set_title('Middle')

ax3.bar(x[len(x)-50], z[len(x)-50], width=1.5)
ax3.set_xlabel('Frequency (Hz)')
ax3.set_ylabel('Magnitude')
ax3.set_title('Almost End')

ax4.bar(x[-1], z[-1], width=1.5)
ax4.set_xlabel('Frequency (Hz)')
ax4.set_ylabel('Magnitude')
ax4.set_title('End')

plt.show()

#### Hilbert Huang Transform Analysis.

##### IMFs Decomposition.

In [ ]:
# Hilbert Huang Transform Analysis
hht_window_size = 2*2560
for bearing in bearings:

    bearing.results['imfs'] = {'h' : imfs_decomposition(bearing.data['vib']['h_acc'], window_size=hht_window_size), 
                               'v' : imfs_decomposition(bearing.data['vib']['v_acc'], window_size=hht_window_size)}

In [ ]:
data = bearings[0].results['imfs']['v']

imf = 5
f, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharey=True, gridspec_kw={'hspace': 1})
ax1.plot(data[0][imf])
ax1.set_ylabel('Magnitude')
ax1.set_title('IMF 0')

ax2.plot(data[len(data)//2][imf])
ax2.set_ylabel('Magnitude')
ax2.set_title('IMF 0')

ax3.plot(data[len(data)-100][imf])
ax3.set_ylabel('Magnitude')
ax3.set_title('IMF 0')

ax4.plot(data[len(data)-1][imf])
ax4.set_ylabel('Magnitude')
ax4.set_title('IMF 0')
plt.show()

In [ ]:
# Plotting IMFs and saving plots.
s_imfs = []
colors = itertools.cycle(palette)
for (i, bearing), color in zip(enumerate(bearings), colors):
    data = bearing.results['imfs']['v']
    n_imfs = min([len(x) for x in data])
    times = [0, len(data)//2, len(data)-100, len(data)-1]
    times_text = ['Beginning', 'Middle', 'Almost End', 'End']
    for imf in range(n_imfs):
        s_imf = []
        for i, t in enumerate(times):
            s = figure(plot_width = 500, plot_height = 500, 
                      title = 'IMF %s. %s.' % (imf, times_text[i]),
                      x_axis_label = 'Recordings', y_axis_label = 'Magnitude')

            x = np.arange(len(data[t][imf])); y = data[t][imf] 
            s.line(x=x, y=y, color=color, legend_label="%s" %(bearing.name))

            s.legend.location = "top_left"

            s_imf.append(s)
        s_imfs.append(s_imf)

for t_imfs in s_imfs:
    y_first_range = t_imfs[0].y_range
    for t_imf in t_imfs:
        t_imf.y_range = y_first_range
    
        
imfs_plot = []
for i in range(4):
    imfs_plot.append([imf[i] for imf in s_imfs])

show(( row(column(imfs_plot[0]), column(imfs_plot[1]), column(imfs_plot[2]), column(imfs_plot[3])) ))
export.export_png(row(column(imfs_plot[0]), column(imfs_plot[1]), column(imfs_plot[2]), column(imfs_plot[3])))

#### Cumsum and Derivative.

In [4]:
# Cumsum and derivative analysis.
for bearing in bearings:
    # Computing cumsum.
    bearing.results['cumsum'] = {'h' : cumsum(bearing.data['vib']['h_acc']), 
                                 'v' : cumsum(bearing.data['vib']['v_acc'])}
    
    """# Computing cumsum derivative.
    h = 39*10**-6 # Distance between points - It's in original data in u-sec column.
    bearing.results['cs_derivative'] = {'h' : np.asarray(derivative(bearing.results['cumsum']['h'].values, h)), 
                                        'v' : np.asarray(derivative(bearing.results['cumsum']['v'].values, h))}
    
    # Marking change points (cp) in derivative.
    bearing.results['cs_deriv_cp'] = {'h' : step_change_point(bearing.results['cs_derivative']['h']),
                                      'v' : step_change_point(bearing.results['cs_derivative']['v'])}"""
                                      

In [5]:
# Plotting cumsum and saving plots.
s_cs = []
colors = itertools.cycle(palette)
for i, bearing in enumerate(bearings):
    
    s = figure(plot_width = 500, plot_height = 500, 
                  title = 'Cumulative Sum.',
                  x_axis_label = 'Recordings', y_axis_label = 'Cumulative Sum')
    
    for (label_l, data), color in zip(bearing.results['cumsum'].items(), colors):
        # Get data points spaced by sample_step. 
        x = np.arange(len(data)//100); data = data[::100]
        # Add circle glyph.
        s.circle(x=x, y=data, color=color, size=1, legend_label="%s, %s" %(bearing.name, label_l))
    
    s.legend.location = "top_left"
    
    s_cs.append(s)
    
show(column(s_cs))

/home/az/.local/lib/python3.7/site-packages/bokeh/models/sources.py:173: BokehUserWarning:

ColumnDataSource's columns must be of the same length. Current lengths: ('x', 71756), ('y', 71757)

/home/az/.local/lib/python3.7/site-packages/bokeh/models/sources.py:173: BokehUserWarning:

ColumnDataSource's columns must be of the same length. Current lengths: ('x', 71756), ('y', 71757)

/home/az/.local/lib/python3.7/site-packages/bokeh/models/sources.py:173: BokehUserWarning:

ColumnDataSource's columns must be of the same length. Current lengths: ('x', 22297), ('y', 22298)

/home/az/.local/lib/python3.7/site-packages/bokeh/models/sources.py:173: BokehUserWarning:

ColumnDataSource's columns must be of the same length. Current lengths: ('x', 22297), ('y', 22298)

/home/az/.local/lib/python3.7/site-packages/bokeh/models/sources.py:173: BokehUserWarning:

ColumnDataSource's columns must be of the same length. Current lengths: ('x', 36556), ('y', 36557)

/home/az/.local/lib/python3.7/site-packa

#### RMS.

In [ ]:
# Starting with fft spectrum analysis.
for bearing in bearings:
    # Computing fft spectogram.
    bearing.results['rms'] = {'h' : rms(bearing.data['vib']['h_acc'].values, window_size=2560), 
                              'v' : rms(bearing.data['vib']['v_acc'].values, window_size=2560)}

In [ ]:
# Plotting cumsum and saving plots.
s_rms = []
colors = itertools.cycle(palette)
for i, bearing in enumerate(bearings):
    
    s = figure(plot_width = 500, plot_height = 500, 
                  title = 'RMS.',
                  x_axis_label = 'Recordings', y_axis_label = 'RMS')
    
    for (label_l, data), color in zip(bearing.results['rms'].items(), colors):
        # Get data points spaced by sample_step. 
        x = np.arange(len(data));
        # Add circle glyph.
        s.circle(x=x, y=data, color=color, size=1, legend_label="%s, %s" %(bearing.name, label_l))
    
    s.legend.location = "top_left"
    
    s_rms.append(s)
    
show(row(column(s_rms), column(s_cs)))

#### Mao et. al. - Correlation coefficient.
<sub>Mao, W., He, J., Tang, J. and Li, Y., 2018. Predicting remaining useful life of rolling bearings based on deep feature representation and long short-term memory neural network. Advances in Mechanical Engineering, 10(12), p.1687814018817184.

In [ ]:
for bearing in bearings:
    # Compute hankel matrix singular values.
    bearing.results['hankel_svd'] = {'h' : hankel_svd(bearing.data['vib']['h_acc'], hankel_window_size=9,
                                                      slice_window_size=len(bearing.data['vib']['h_acc'])//2560),
                                     
                                     'v' : hankel_svd(bearing.data['vib']['v_acc'], hankel_window_size=9, 
                                                      slice_window_size=len(bearing.data['vib']['v_acc'])//2560)}
    
    # Compute correlation coefficients.
    bearing.results['hankel_svd_correlation_coeffs'] = {'h' : correlation_coeffs(bearing.results['hankel_svd']['h'], 
                                                        baseline_percentage=20, norm_interval=[-1, 1]),
                                                  
                                                        'v' : correlation_coeffs(bearing.results['hankel_svd']['v'],
                                                        baseline_percentage=20, norm_interval=[-1, 1])}

In [ ]:
# Plotting Hankel matrix singular values correlation coefficients.
s_h_svd = []
colors = itertools.cycle(palette)
for bearing in bearings:
    s = figure(plot_width = 500, plot_height = 500, 
           title = 'Hankel matrix singular values correlation coefficients.',
           x_axis_label = 'Recordings', y_axis_label = 'Correlation coefficients')
    
    for (label_l, data), color in zip(bearing.results['hankel_svd_correlation_coeffs'].items(), colors):
        x = np.arange(len(data))
        # Add circle glyph.
        s.circle(x=x, y=data, color=color, legend_label='%s, %s' % (bearing.name, label_l))
    
    s.legend.location = 'bottom_left'
    
    s_h_svd.append(s)

show(row(column(s_h_svd)))

#### PCA
<sub> https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
pca = PCA(n_components=2)
for bearing in bearings:
    # Adjusting shape and zero mean.
    h = np.squeeze(StandardScaler(with_mean=True, with_std=True).fit_transform(bearing.results['cumsum']['h'].values.reshape(-1 ,1)))
    v = np.squeeze(StandardScaler(with_mean=True, with_std=True).fit_transform(bearing.results['cumsum']['v'].values.reshape(-1 ,1)))
    
    # Adding np.arange column and compute PCA.
    bearing.results['cs_pca'] = {'h' : pca.fit_transform(list(zip(np.arange(len(h)), h))), 
                                 'v' : pca.fit_transform(list(zip(np.arange(len(v)), v)))} 

In [ ]:
# Plotting pca components.
s_pca = []
colors = itertools.cycle(palette)
for bearing in bearings:
    
    s = figure(plot_width = 500, plot_height = 500, 
           title = 'PCA Components.',
           x_axis_label = 'Component 1', y_axis_label = 'Component 2')
    
    for (label_l, data), color in zip(bearing.results['cs_pca'].items(), colors):
        # Get each data point after sample_step. 
        data = data[::2560]
        # Add circle glyph.
        s.circle(x=data[:, 0], y=data[:, 1], color=color, legend_label="%s, %s" %(bearing.name, label_l))

    s.legend.location = "top_left"
    
    s_pca.append(s)
    
show(row(column(s_pca), column(s_cs)))

### Plotting all results

In [ ]:
# Plotting all results.  
show((row(column(s_cs), column(s_deriv), column(s_h_svd), column(s_pca))))

### Saving results.

In [6]:
# Saving results to binary.
for bearing in bearings:
    bearing.save_r()

# Exporting all results to image.
# https://docs.bokeh.org/en/latest/docs/user_guide/export.html?highlight=export
export.export_png(row(column(s_cs)))

'/tmp/tmpd38nyegp.png'

### RUL Estimation